# Script to Consolidate Data and Create a Day Count

In [34]:
import pandas as pd
import glob

In [35]:
path = "D:/Dropbox/Data/R Shiny/20-03-20 Covid-19"

## Combine all the files together

In [36]:
all_files = glob.glob(path + "/csse_covid_19_daily_reports/*.csv")

li = []

# for files in folder, create a dataframe out of them
for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    # create a date field that is generated from the file name
    df['date'] = filename.split('\\')[-1].split('.')[0]
    # If the file has the new headers, replace it with the old header format for Country/Region
    df = df.rename(columns={'Country_Region': 'Country/Region','Lat':'Latitude','Long':'Longitude'})
    # add data to list
    li.append(df)

# concatinate all data (union) into final dataframe    
frame = pd.concat(li, axis=0, ignore_index=True)

# Change Country Names for consistency
frame["Country/Region"].replace({"occupied Palestinian territory": "Palestine", "Mainland China": "China", "Taiwan*": "Taipei and environs", "UK": "United Kingdom", "The Bahamas": "Bahamas, The", "Iran (Islamic Republic of)": "Iran", "Hong Kong SAR": "Hong Kong", "The Gambia": "Gambia, The", "Gambia": "Gambia, The", "Viet Nam": "Vietnam", "Korea, South": "South Korea", "Cape Verde": "Cabo Verde", "Czechia": "Czech Republic", "Ivory Coast": "Cote d'Ivoire", "Timor-Leste": "East Timor", "St. Martin": "Saint Martin", "Bahamas": "Bahamas, The", "Macao SAR":"Macau","West Bank and Gaza":"Palestine","Taiwan":"Taipei and environs","Republic of Ireland":"Ireland","Republic of Korea":"South Korea","Republic of Moldova":"Moldova","Republic of the Congo":"Congo (Brazzaville)","Russian Federation":"Russia"}, inplace=True)

In [37]:
# aggregate numbers by country
country_df = frame.groupby(['Country/Region','date']).sum().reset_index()

## Aggregate the numbers on a country level

In [38]:
country_list = []

# split the dataframe by country
for country, country_df in country_df.groupby('Country/Region'):
    # Add a day field and number it chronologically
    country_df = country_df.reset_index()
    country_df['Days'] = country_df.index + 1
    
    # Get deltas for deaths, recovered, and then new cases using pd.diff() function
    country_df['New_Deaths'] = country_df['Deaths'].diff()
    country_df['New_Recovered'] = country_df['Recovered'].diff()
    country_df['New_Confirmed'] = country_df['Confirmed'].diff()
    
    
    # add data to list
    country_list.append(country_df)
    
# concatinate all data (union) into final dataframe    
frame2 = pd.concat(country_list, axis=0, ignore_index=True)

In [39]:
# Ensure new values are > 0
import numpy as np
frame2['New_Confirmed'] = np.where(frame2['New_Confirmed'] < 0, 0, frame2['New_Confirmed'])
frame2['New_Deaths'] = np.where(frame2['New_Deaths'] < 0, 0, frame2['New_Deaths'])
frame2['New_Recovered'] = np.where(frame2['New_Recovered'] < 0, 0, frame2['New_Recovered'])

## Cleaning the data

In [40]:
# delete newly created fields
del frame2['index']

In [41]:
# rearrange fields for easier reading
final_table = frame2[['date','Days','Country/Region','Confirmed','New_Confirmed','Deaths','New_Deaths','Recovered','New_Recovered']]
#replace NaN with empty string
final_table = final_table.fillna(0)
# Get active cases for the day
final_table['Active_Cases'] = final_table['Confirmed'] - final_table['Deaths'] - final_table['Recovered']

In [42]:
# for row in dataframe
for i, row in final_table.iterrows():
    # if Days == 1
    if (row['Days'] == 1):
        # C_delta == Confirmed, D_delta == Deaths, R_delta == Recovered
        final_table.at[i,'New_Confirmed'] = row['Confirmed']
        final_table.at[i,'New_Deaths'] = row['Deaths']
        final_table.at[i,'New_Recovered'] = row['Recovered']

In [43]:
final_table[final_table['Country/Region'] == 'US']

date  Days Country/Region  Confirmed  New_Confirmed  Deaths  \
4580  01-22-2020     1             US        1.0            1.0     0.0   
4581  01-23-2020     2             US        1.0            0.0     0.0   
4582  01-24-2020     3             US        2.0            1.0     0.0   
4583  01-25-2020     4             US        2.0            0.0     0.0   
4584  01-26-2020     5             US        5.0            3.0     0.0   
4585  01-27-2020     6             US        5.0            0.0     0.0   
4586  01-28-2020     7             US        5.0            0.0     0.0   
4587  01-29-2020     8             US        5.0            0.0     0.0   
4588  01-30-2020     9             US        5.0            0.0     0.0   
4589  01-31-2020    10             US        6.0            1.0     0.0   
4590  02-01-2020    11             US        8.0            2.0     0.0   
4591  02-02-2020    12             US        8.0            0.0     0.0   
4592  02-03-2020    13             US       11.0            3.0     0.0   
4593  02-04-2020    14             US       11.0            0.0     0.0   
4594  02-05-2020    15             US       12.0            1.0     0.0   
4595  02-06-2020    16             US       12.0            0.0     0.0   
4596  02-07-2020    17             US       12.0            0.0     0.0   
4597  02-08-2020    18             US       12.0            0.0     0.0   
4598  02-09-2020    19             US       12.0            0.0     0.0   
4599  02-10-2020    20             US       12.0            0.0     0.0   
4600  02-11-2020    21             US       13.0            1.0     0.0   
4601  02-12-2020    22             US       13.0            0.0     0.0   
4602  02-13-2020    23             US       15.0            2.0     0.0   
4603  02-14-2020    24             US       15.0            0.0     0.0   
4604  02-15-2020    25             US       15.0            0.0     0.0   
4605  02-16-2020    26             US       15.0            0.0     0.0   
4606  02-17-2020    27             US       15.0            0.0     0.0   
4607  02-18-2020    28             US       15.0            0.0     0.0   
4608  02-19-2020    29             US       15.0            0.0     0.0   
4609  02-20-2020    30             US       15.0            0.0     0.0   
...          ...   ...            ...        ...            ...     ...   
4617  02-28-2020    38             US       62.0            2.0     0.0   
4618  02-29-2020    39             US       70.0            8.0     1.0   
4619  03-01-2020    40             US       76.0            6.0     1.0   
4620  03-02-2020    41             US      101.0           25.0     6.0   
4621  03-03-2020    42             US      122.0           21.0     7.0   
4622  03-04-2020    43             US      153.0           31.0    11.0   
4623  03-05-2020    44             US      221.0           68.0    12.0   
4624  03-06-2020    45             US      278.0           57.0    14.0   
4625  03-07-2020    46             US      417.0          139.0    17.0   
4626  03-08-2020    47             US      537.0          120.0    21.0   
4627  03-09-2020    48             US      605.0           68.0    22.0   
4628  03-10-2020    49             US      959.0          354.0    28.0   
4629  03-11-2020    50             US     1281.0          322.0    36.0   
4630  03-12-2020    51             US     1663.0          382.0    40.0   
4631  03-13-2020    52             US     2179.0          516.0    47.0   
4632  03-14-2020    53             US     2726.0          547.0    54.0   
4633  03-15-2020    54             US     3499.0          773.0    63.0   
4634  03-16-2020    55             US     4632.0         1133.0    85.0   
4635  03-17-2020    56             US     6421.0         1789.0   108.0   
4636  03-18-2020    57             US     7786.0         1365.0   118.0   
4637  03-19-2020    58             US    13680.0         5894.0   200.0   
4638  03-20-2020    59             US

In [44]:
final_table.to_csv(path + "/transformed.csv", index=False)